In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import pickle
from sklearn.metrics import accuracy_score

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = 'cpu'

In [3]:
# Define the dataset
class LungDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.labels = {'pneumonia': 0, 'covid': 1, 'normal': 2}  # labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        e = torch.stack(item['e'])  # Stack tensors to form a single tensor
        # Normalize the tensor between 0 and 1
        e_min = e.min()
        e_max = e.max()
        e = (e - e_min) / (e_max - e_min)
        label = self.labels[item['label']]
        return e, label

# Load your dataset
data = []
for file in os.listdir(data_dir):
    f = open(os.path.join(data_dir, file), 'rb')
    data.append(pickle.load(f))
dataset = LungDataset(data)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)





In [9]:
# Define the model
class LungClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LungClassifier, self).__init__()
        self.fc = nn.Linear(input_size, num_classes, bias=False)

    def forward(self, x):
        out = self.fc(x)
        return F.log_softmax(out, dim=1)

input_size = 60
num_classes = 3
model = LungClassifier(input_size, num_classes).to(device)

# Training setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

In [22]:
# Training loop
num_epochs = 400  

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    all_labels = []
    all_predictions = []
    
    for e, labels in dataloader:
        e, labels = e.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(e)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # Collect predictions and labels for accuracy calculation
        _, predicted = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())
    
    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = accuracy_score(all_labels, all_predictions)

    scheduler.step(epoch_loss)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

Epoch [1/400], Loss: 1.0948, Accuracy: 0.3700
Epoch [2/400], Loss: 1.0855, Accuracy: 0.4100
Epoch [3/400], Loss: 1.0794, Accuracy: 0.4300
Epoch [4/400], Loss: 1.0748, Accuracy: 0.4400
Epoch [5/400], Loss: 1.0712, Accuracy: 0.4500
Epoch [6/400], Loss: 1.0681, Accuracy: 0.4550
Epoch [7/400], Loss: 1.0653, Accuracy: 0.4570
Epoch [8/400], Loss: 1.0630, Accuracy: 0.4600
Epoch [9/400], Loss: 1.0609, Accuracy: 0.4650
Epoch [10/400], Loss: 1.0585, Accuracy: 0.4675
Epoch [11/400], Loss: 1.0567, Accuracy: 0.4700
Epoch [12/400], Loss: 1.0549, Accuracy: 0.4740
Epoch [13/400], Loss: 1.0534, Accuracy: 0.4760
Epoch [14/400], Loss: 1.0520, Accuracy: 0.4780
Epoch [15/400], Loss: 1.0505, Accuracy: 0.4800
Epoch [16/400], Loss: 1.0491, Accuracy: 0.4815
Epoch [17/400], Loss: 1.0480, Accuracy: 0.4820
Epoch [18/400], Loss: 1.0469, Accuracy: 0.4850
Epoch [19/400], Loss: 1.0459, Accuracy: 0.4870
Epoch [20/400], Loss: 1.0450, Accuracy: 0.4900
Epoch [21/400], Loss: 1.0440, Accuracy: 0.4920
Epoch [22/400], Loss: 

In [24]:
weights = model.fc.weight.data.cpu()  
torch.save(weights, 'learned_weights.pt')